In [2]:
import cv2 as cv
import numpy as np
import os
import matplotlib.pyplot as plt

In [3]:
def show_image(title, image):
    image = cv.resize(image, (0, 0), fx = 1, fy = 1)
    image_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    plt.imshow(image_rgb)
    plt.title(title)
    plt.axis('off')
    plt.show()

In [4]:
def extract_table(image):
    hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)

    lower_blue = np.array([90, 50, 50])
    upper_blue = np.array([130, 255, 255])
    mask = cv.inRange(hsv, lower_blue, upper_blue)

    kernel = np.ones((5, 5), np.uint8)
    mask_cleaned = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)
    mask_cleaned = cv.morphologyEx(mask_cleaned, cv.MORPH_OPEN, kernel)
    
    edges = cv.Canny(mask_cleaned, 50, 150)
    contours, _ = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    max_area = 0
    top_left = top_right = bottom_left = bottom_right = None
    
    for contour in contours:
        if len(contour) > 3:
            possible_top_left = possible_bottom_right = None
            for point in contour.squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point
                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1]:
                    possible_bottom_right = point

            diff = np.diff(contour.squeeze(), axis=1)
            possible_top_right = contour.squeeze()[np.argmin(diff)]
            possible_bottom_left = contour.squeeze()[np.argmax(diff)]
            
            area = cv.contourArea(np.array([possible_top_left, possible_top_right, possible_bottom_right, possible_bottom_left]))
            if area > max_area:
                max_area = area
                top_left = possible_top_left
                top_right = possible_top_right
                bottom_left = possible_bottom_left
                bottom_right = possible_bottom_right

    if any(corner is None for corner in [top_left, top_right, bottom_left, bottom_right]):
        raise ValueError("Unable to detect the table. Please check the input image.")

    width, height = 940, 940
    
    puzzle = np.array([top_left, top_right, bottom_left, bottom_right], dtype="float32")
    destination = np.array([[0, 0], [width, 0], [0, height], [width, height]], dtype="float32")
    M = cv.getPerspectiveTransform(puzzle, destination)
    result = cv.warpPerspective(image, M, (width, height))

    return result

In [5]:
def crop_board(image, corners):
    width, height = 1400, 1400 # 14 x 100
    src_points = np.array(corners, dtype="float32")
    dst_points = np.array([[0, 0], [width, 0], [0, height], [width, height]], dtype = "float32")
    M = cv.getPerspectiveTransform(src_points, dst_points)
    warped = cv.warpPerspective(image, M, (width, height), flags=cv.INTER_LINEAR)
    
    sharpened = cv.filter2D(warped, -1, np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]))
    clearer_image = cv.bilateralFilter(sharpened, d = 9, sigmaColor = 75, sigmaSpace = 75)

    return clearer_image

In [6]:
lines_horizontal = []
for i in range(0, 1401, 100):
    l = []
    l.append((0, i))
    l.append((1399, i))
    lines_horizontal.append(l)

lines_vertical = []
for i in range(0, 1401, 100):
    l = []
    l.append((i, 0))
    l.append((i, 1399))
    lines_vertical.append(l)

In [ ]:
image = cv.imread('training_data/3_15.jpg')
show_image("The image", image)

board = extract_table(image)
show_image("The game board", board)

corners = [(125, 123), (816, 125), (121, 819), (816, 822)]
cropped_board = crop_board(board, corners)
show_image("The cropped game board", cropped_board)

for line in  lines_vertical : 
    cv.line(cropped_board, line[0], line[1], (0, 255, 0), 3)
    for line in  lines_horizontal : 
        cv.line(cropped_board, line[0], line[1], (0, 0, 255), 3)

show_image('The cells delimited by the horizontal and vertical lines', cropped_board)

## Task 1 functions and parameters:

In [8]:
corners = [(125, 123), (817, 125), (121, 819), (818, 822)]
rows = 14
cols = 14
board_size = 1400
line_width = 7

letters = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N"]
games = ["1", "2", "3", "4"]

folder_path = 'antrenare'

In [9]:
def detect_added_piece(initial_board, current_board, rows = 14, cols = 14, board_size = 1400, line_width = 7):

    cell_size = (board_size - rows * line_width) // rows
    sum_max = 0
    chosen_row, chosen_column = -1, -1

    for row in range(rows):
        for col in range(cols):
            x_start = col * (cell_size + line_width)
            x_end = x_start + cell_size
            y_start = row * (cell_size + line_width)
            y_end = y_start + cell_size

            initial_cell = initial_board[y_start:y_end, x_start:x_end]
            current_cell = current_board[y_start:y_end, x_start:x_end]

            diff_sum = np.sum(np.abs(initial_cell.astype(int) - current_cell.astype(int)))

            if diff_sum > sum_max:
                sum_max = diff_sum
                chosen_row = row
                chosen_column = col
    
    return (chosen_row, chosen_column)


## Task 2 functions and parameters:

In [10]:
numbers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 24, 25, 
           27, 28, 30, 32, 35, 36, 40, 42, 45, 48, 49, 50, 54, 56, 60, 63, 64, 70, 72, 80, 81, 90]

In [11]:
def select_cell(row, col, board):
    cell_size = (board_size - rows * line_width) // rows

    x_start = col * (cell_size + line_width)
    x_end = x_start + cell_size
    y_start = row * (cell_size + line_width)
    y_end = y_start + cell_size
    
    cell = board[y_start:y_end, x_start:x_end]

    return cell

In [12]:
def number_classification(patch, numbers=[]):
    maxi = -np.inf
    poz = -1
    template = None
    
    for j in range(len(numbers)):
        img_template = cv.imread(f'templates/{j}.jpg', cv.IMREAD_GRAYSCALE)

        corr = cv.matchTemplate(patch, img_template, cv.TM_CCOEFF_NORMED)
        corr = np.max(corr)

        if corr > maxi:
            maxi = corr
            poz = j
            template = img_template
        
    return (poz, template)

In [13]:
def remove_margins(binary_image, max_margin_lines = 10, margin_color = 0):
    initial_size = binary_image.shape[:2]

    def is_partially_color(line, color):
        if color == 0:
            return np.mean(line) < 230
        else:
            return np.mean(line) > 25

    for i in range(max_margin_lines):
        if is_partially_color(binary_image[0, :], margin_color):
            binary_image = binary_image[1:, :]
        else:
            break

    for i in range(max_margin_lines):
        if is_partially_color(binary_image[-1, :], margin_color):
            binary_image = binary_image[:-1, :]
        else:
            break

    for i in range(max_margin_lines):
        if is_partially_color(binary_image[:, 0], margin_color):
            binary_image = binary_image[:, 1:]
        else:
            break

    for i in range(max_margin_lines):
        if is_partially_color(binary_image[:, -1], margin_color):
            binary_image = binary_image[:, :-1]
        else:
            break

    processed_image = cv.resize(binary_image, (initial_size[1], initial_size[0]), interpolation = cv.INTER_NEAREST)

    return processed_image

In [14]:
def add_border(image, border_width = 10, border_color = 255):
    height, width = image.shape
    padded_image = np.ones((height, width + 2 * border_width), dtype=np.uint8) * border_color
    
    padded_image[:, border_width:border_width + width] = image
    
    return padded_image

## Template making

In [42]:
def extract_cells(board_image, lines_horizontal, lines_vertical):
    cells = []
    for i in range(5, 11):
        for j in range(4, 12):
            y_min = lines_vertical[j][0][0] + 5
            y_max = lines_vertical[j + 1][1][0] - 5
            x_min = lines_horizontal[i][0][1] + 5
            x_max = lines_horizontal[i + 1][1][1] - 5
            cell = board_image[x_min:x_max, y_min:y_max].copy()
            cells.append(cell)
          
    return cells

In [44]:
MIN_HEIGHT = 20
MAX_HEIGHT = 100
MIN_WIDTH = 20
MAX_WIDTH = 100

output_folder = "templates_1/"
corners = [(125, 123), (817, 125), (121, 819), (818, 822)]

board_image = cv.imread('auxiliary_images/03.jpg')
board_image = extract_table(board_image)
board_image = crop_board(board_image, corners)

cells = extract_cells(board_image, lines_horizontal = lines_horizontal, lines_vertical = lines_vertical)

for i in range(len(cells)):
    cell = cv.cvtColor(cells[i], cv.COLOR_BGR2GRAY)
    _, thresh = cv.threshold(cell, 100, 255, cv.THRESH_BINARY)
    img = remove_margins(thresh, max_margin_lines = 10, margin_color = 0) # black

    dst = cv.Canny(img, 0, 150)
    blured = cv.blur(dst, (5, 5), 0)

    MIN_CONTOUR_AREA = 200

    img_thresh = cv.adaptiveThreshold(blured, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV, 11, 2)

    Contours, imgContours = cv.findContours(img_thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)

    for contour in Contours:
        if cv.contourArea(contour) > MIN_CONTOUR_AREA:
            [X, Y, W, H] = cv.boundingRect(contour)

            if MIN_HEIGHT <= H <= MAX_HEIGHT and MIN_WIDTH <= W <= MAX_WIDTH:
                cropped_image = thresh[Y:Y + H, X:X + W]
    
    if i >= 0 and i <= 9:
        cropped_image = add_border(cropped_image, border_width = 15, border_color = 255)
        output_path = os.path.join(output_folder, f"{i}.jpg")
        cv.imwrite(output_path, cropped_image)

    if i >= 10 and i <= 45:
        output_path = os.path.join(output_folder, f"{i}.jpg")
        cv.imwrite(output_path, cropped_image)

## Task 3 functions and parameters:

In [19]:
# change the path to the path where you will find the images of the game
data_path = "training_data/"

# add the number for each game - for example if there are 3 games the list should look like this -> games = ["1", "2", "3"]
games = ["1", "2", "3", "4"]

In [16]:
def define_matrix(board): 
    special_cells = {
        (0, 0): "3x", (0, 6): "3x", (0, 7): "3x", (0, 13): "3x", (6, 0): "3x", (6, 13): "3x", (7, 0): "3x", (7, 13): "3x", (13, 0): "3x", (13, 6): "3x", (13, 7): "3x", (13, 13): "3x", # celule cu "3x"
        (1, 1): "2x", (1, 12): "2x", (2, 2): "2x", (2, 11): "2x", (3, 3): "2x", (3, 10): "2x", (4, 4): "2x", (4, 9): "2x", (9, 4): "2x", (9, 9): "2x", (10, 3): "2x", (10, 10): "2x", (11, 3): "2x", (11, 11): "2x", (12, 1): "2x", (12, 12): "2x", # celule cu "2x"
        (6, 6): "1", (6, 7): "2", (7, 6): "3", (7, 7): "4", # celule cu numere "1, 2, 3, 4"
        (3, 6): "+", (4, 7): "+", (6, 4): "+", (6, 10): "+", (7, 3): "+", (7, 9): "+", (9, 6): "+", (10, 7): "+", # celule cu semnul adunarii "+"
        (2, 5): "-", (2, 8): "-", (5, 2): "-", (5, 11): "-", (8, 2): "-", (8, 11): "-", (11, 5): "-", (11, 8): "-", # celule cu semnul scaderii "-"
        (3, 7): "*", (4, 6): "*", (6, 3): "*", (6, 9): "*", (7, 4): "*", (7, 10): "*", (9, 7): "*", (10, 6): "*", # celule cu semnul inmultirii "*"
        (1, 4): ":", (1, 9): ":", (4, 1): ":", (4, 12): ":", (9, 1): ":", (9, 12): ":", (12, 4): ":", (12, 9): ":" # celule cu semnul impartirii ":"
    }

    for (row, col), cell_type in special_cells.items():
        board[row][col] = cell_type
    
    return board

In [17]:
def get_turns(game):
    turns_path = data_path + game + "_turns.txt"
    lines = []
    
    with open(turns_path, "r") as file:
        all_lines = file.readlines()
    
    for i, line in enumerate(all_lines):
        if i < len(all_lines) - 1:
            processed_line = line[:-1].split()
        else:
            processed_line = line.split()
        
        lines.append(processed_line)

    return lines

In [20]:
all_special_cells = ["empty", "3x", "2x", "+", "-", "*", ":"]
cells_with_signs = ["+", "-", "*", ":"]

# Aici se gaseste folder-ul cu solutii
output_file_path = "evaluation/solutions_train/" 

for game in games:
    images = sorted([f for f in os.listdir(data_path) if f.startswith(f'{game}_') and f.endswith('.jpg')])

    board = [["empty" for _ in range(14)] for _ in range(14)]
    define_matrix(board)
    
    turns = get_turns(game)

    player = turns[0][0]
    first_round = 0

    turn = 1

    output_file_task_3_turns = output_file_path + f"{game}_turns.txt"
    output_file_task_3_scores = output_file_path + f"{game}_scores.txt"

    while turn <= len(turns):
        score = 0
        if turn < len(turns):
            last_round = int(turns[turn][1]) - 1
        else:
            last_round = len(images)
            
        for index in range(first_round, last_round):
            if index >= 0 and index <= 8:
                output_file_task_1_2 = output_file_path + f"{game}_0{index + 1}.txt"
            else:
                output_file_task_1_2 = output_file_path + f"{game}_{index + 1}.txt"

            if index == 0:
                image1_path = "auxiliary_images/01.jpg"
                image2_path = os.path.join(data_path, images[index])
            else:
                image1_path = os.path.join(data_path, images[index - 1])
                image2_path = os.path.join(data_path, images[index])
            
            image1 = cv.imread(image1_path)
            image2 = cv.imread(image2_path)
            
            board1 = extract_table(image1)
            board1 = crop_board(board1, corners)
            
            board2 = extract_table(image2)
            board2 = crop_board(board2, corners)
            
            added_piece_coordinates = detect_added_piece(board1, board2, rows, cols, board_size, line_width)

            i = added_piece_coordinates[0] # row
            j = added_piece_coordinates[1] # column

            cell = select_cell(i, j, board2)
            cell = cv.cvtColor(cell, cv.COLOR_BGR2GRAY)
            _, cell = cv.threshold(cell, 100, 255, cv.THRESH_BINARY)

            cell = add_border(cell, border_width = 15, border_color = 255)

            (number, template) = number_classification(cell, numbers = numbers)

            number = numbers[number]

            initial_matrix_cell = board[i][j]

            board[i][j] = number

            with open(output_file_task_1_2, 'a') as file:
                if added_piece_coordinates != None:
                    line = str(i + 1) + str(letters[j]) + " " + str(number)
                else:
                    line = "None"
                file.write(line + '\n')
            
            if j + 1 <= 13 and j + 2 <= 13:
                if board[i][j + 1] not in all_special_cells and board[i][j + 2] not in all_special_cells:
                    right1 = int(board[i][j + 1])
                    right2 = int(board[i][j + 2])

                    if initial_matrix_cell in cells_with_signs:
                        if initial_matrix_cell == "+":
                            if right1 + right2 == number:
                                score = score + number

                        elif initial_matrix_cell == "-":
                            if right1 - right2 == number or right2 - right1 == number:
                                score = score + number

                        elif initial_matrix_cell == "*":
                            if right1 * right2 == number:
                                score = score + number

                        elif initial_matrix_cell == ":":
                            if right1 == 0 and right2 == 0:
                                score = score
                            elif right1 == 0 and right1 / right2 == number:
                                score = score + number
                            elif right2 == 0 and right2 / right1 == number:
                                score = score + number
                            elif right1 / right2 == number or right2 / right1 == number:
                                score = score + number 
                    else:
                        if (right1 + right2 == number) or (right1 - right2 == number) or (right2 - right1 == number) or (right1 * right2 == number) or (right1 != 0 and right2 != 0 and right1 / right2 == number) or (right1 != 0 and right2 !=0 and right2 / right1 == number) or (right2 != 0 and right1 / right2 == number) or (right1 != 0 and right2 / right1 == number):
                            if initial_matrix_cell == "2x":
                                score = score + number * 2
                            elif initial_matrix_cell == "3x":
                                score = score + number * 3
                            else:
                                score = score + number
                            
            if j - 1 >= 0 and j - 2 >= 0:
                if board[i][j - 1] not in all_special_cells and board[i][j - 2] not in all_special_cells:
                    left1 = int(board[i][j - 1])
                    left2 = int(board[i][j - 2])

                    if initial_matrix_cell in cells_with_signs:
                        if initial_matrix_cell == "+":
                            if left1 + left2 == number:
                                score = score + number

                        elif initial_matrix_cell == "-":
                            if left1 - left2 == number or left1 - left2 == number:
                                score = score + number

                        elif initial_matrix_cell == "*":
                            if left1 * left2 == number:
                                score = score + number

                        elif initial_matrix_cell == ":":
                            if left1 == 0 and left2 == 0:
                                score = score
                            elif left1 == 0 and left1 / left2 == number:
                                score = score + number
                            elif left2 == 0 and left2 / left1 == number:
                                score = score + number
                            elif left1 / left2 == number or left2 / left1 == number:
                                score = score + number 
                    else:
                        if (left1 + left2 == number) or (left1 - left2 == number) or (left2 - left1 == number) or (left1 * left2 == number) or (left1 != 0 and left2 != 0 and left1 / left2 == number) or (left1 != 0 and left2 !=0 and left2 / left1 == number) or (left2 != 0 and left1 / left2 == number) or (left1 != 0 and left2 / left1 == number):
                            if initial_matrix_cell == "2x":
                                score = score + number * 2
                            elif initial_matrix_cell == "3x":
                                score = score + number * 3
                            else:
                                score = score + number

            if i + 1 <= 13 and i + 2 <= 13:
                if board[i + 1][j] not in all_special_cells and board[i + 2][j] not in all_special_cells:
                    down1 = int(board[i + 1][j])
                    down2 = int(board[i + 2][j])

                    if initial_matrix_cell in cells_with_signs:
                        if initial_matrix_cell == "+":
                            if down1 + down2 == number:
                                score = score + number

                        elif initial_matrix_cell == "-":
                            if down1 - down2 == number or down2 - down1 == number:
                                score = score + number

                        elif initial_matrix_cell == "*":
                            if down1 * down2 == number:
                                score = score + number

                        elif initial_matrix_cell == ":":
                            if down1 == 0 and down2 == 0:
                                score = score
                            elif down1 == 0 and down1 / down2 == number:
                                score = score + number
                            elif down2 == 0 and down2 / down1 == number:
                                score = score + number
                            elif down1 / down2 == number or down2 / down1 == number:
                                score = score + number 
                    else:
                        if (down1 + down2 == number) or (down1 - down2 == number) or (down2 - down1 == number) or (down1 * down2 == number) or (down1 != 0 and down2 != 0 and down1 / down2 == number) or (down1 != 0 and down2 !=0 and down2 / down1 == number) or (down2 != 0 and down1 / down2 == number) or (down1 != 0 and down2 / down1 == number):
                            if initial_matrix_cell == "2x":
                                score = score + number * 2
                            elif initial_matrix_cell == "3x":
                                score = score + number * 3
                            else:
                                score = score + number

            if i - 1 >= 0 and i - 2 >= 0:
                if board[i - 1][j] not in all_special_cells and board[i - 2][j] not in all_special_cells:
                    up1 = int(board[i - 1][j])
                    up2 = int(board[i - 2][j])

                    if initial_matrix_cell in cells_with_signs:
                        if initial_matrix_cell == "+":
                            if up1 + up2 == number:
                                score = score + number

                        elif initial_matrix_cell == "-":
                            if up1 - up2 == number or up2 - up1 == number:
                                score = score + number

                        elif initial_matrix_cell == "*":
                            if up1 * up2 == number:
                                score = score + number

                        elif initial_matrix_cell == ":":
                            if up1 == 0 and up2 == 0:
                                score = score
                            elif up1 == 0 and up1 / up2 == number:
                                score = score + number
                            elif up2 == 0 and up2 / up1 == number:
                                score = score + number
                            elif up1 / up2 == number or up2 / up1 == number:
                                score = score + number 
                    else: 
                        if (up1 + up2 == number) or (up1 - up2 == number) or (up2 - up1 == number) or (up1 * up2 == number) or (up1 != 0 and up2 != 0 and up1 / up2 == number) or (up1 != 0 and up2 !=0 and up2 / up1 == number) or (up2 != 0 and up1 / up2 == number) or (up1 != 0 and up2 / up1 == number):
                            if initial_matrix_cell == "2x":
                                score = score + number * 2
                            elif initial_matrix_cell == "3x":
                                score = score + number * 3
                            else:
                                score = score + number
                        
        with open(output_file_task_3_scores, 'a') as file_scores:
            if added_piece_coordinates != None:
                line = player + " " + str(first_round + 1) + " " + str(score)
            else:
                line = "None"
            file_scores.write(line + '\n')

        with open(output_file_task_3_turns, 'a') as file_turns:
            if added_piece_coordinates != None:
                line = player + " " + str(first_round + 1)
            else:
                line = "None"
            file_turns.write(line + '\n')

        first_round = last_round 
        turn = turn + 1
        if turn <= len(turns):
            player = turns[turn - 1][0]
        